<p style="font-size:200%;font-weight:bold"> Return </p>

- Author: Yichen Luo
- Date: 20220820

In [1]:
import time
import random
import numpy as np
import pandas as pd
from datetime import timezone
import datetime
import tweepy
import requests
import configparser
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re
import jellyfish
import os
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell
from pycoingecko import CoinGeckoAPI
pd.set_option('display.precision', 8)

cg = CoinGeckoAPI()
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
RD = '/home/team_defi/02_rdata/'
WD = '/home/team_defi/03_wdata/'
RES = '/home/team_defi/04_result/'

# Token List

## Duplicates

In [2]:
df = pd.read_csv(f'{RES}/insider_trading/token_related_txn/token_related_txn_without_lp.csv')
df['insider_contractAddress'].nunique()
df['insider_tokenSymbol'].nunique()
df['insider_tokenName'].nunique()

2660

2474

2579

## Generate List

In [4]:
lst = df.drop_duplicates('insider_contractAddress').loc[:, ['insider_tokenSymbol', 'insider_tokenName', 'insider_contractAddress', 'chain']]
lst.sample(3)

,insider_tokenSymbol,insider_tokenName,insider_contractAddress,chain
2012,GFInu,@GonFreecsInu,0x9320a9571252f6359Fcd43Eb4bd1395eb2101ce9,ethereum
4077,CAP,CashApes,0xE2Ac0Afb5C9ca6336d935FC70677Af405c1eD4B3,ethereum
2774,PANDORI,Panda Inu,0x8D54F0cE9719a22336EAd45f885D94b4Cc7864c8,ethereum


In [5]:
lst['status'] = -1
lst
lst.to_csv(f'{WD}/price/list/token_list_complete.csv', index=False)

,insider_tokenSymbol,insider_tokenName,insider_contractAddress,chain,status
0,TKING,Tiger King,0x24e89bdf2f65326b94e36978a7edeac63623dafa,ethereum,-1
1,TRIBE,TRIBE,0x4ffcad848248d4931374336fcaccf263638413ae,ethereum,-1
2,INCH,INCH TOKEN,0x6339bd92eaa3cb8e629b06449e08b90581a33cea,ethereum,-1
6,BITA,BitAstir Token,0x284b19a5fa65f706422c0c86566e407b0e70e537,ethereum,-1
7,TSGO,The Show Goes On,0xab68e9f518620cc76d28fa08f42d4f2939d40f02,ethereum,-1
...,...,...,...,...,...
14544,ROGUE,ROGUE COIN,0x835dF131C80cf4b27cEfdE256498E363cF8a4C27,polygon,-1
14548,OKLP,OkLetsPlay,0x5D48a5E5a3e737322Ae27e25897F1c9E19EcC941,polygon,-1
14549,NXTT,NextEarthToken,0x0d0B8488222F7f83B23E365320a4021b12eAD608,polygon,-1
14639,DGH,Dogioh,0x87842eD511C255a06aE872E7Ad83a64a1e3bD39c,polygon,-1


# CoinGecko

In [6]:
token = pd.read_csv(f'{WD}/price/list/token_list_complete.csv')
coin = pd.read_csv(f'{WD}/price/list/CoinGecko Token API List - CoinGecko Token API List.csv')
dfs = []

for idx, row in token.loc[token['status'] == -1].iterrows():
    try:
        time.sleep(2)
        insider_contractAddress = row['insider_contractAddress']
        insider_tokenSymbol = row['insider_tokenSymbol']
        insider_tokenName = row['insider_tokenName']
        chain = row['chain']

        if chain == 'ethereum': assetplatform = 'ethereum'
        if chain == 'binance': assetplatform = 'binance-smart-chain'
        if chain == 'avalanche': assetplatform = 'avalanche'
        if chain == 'fantom': assetplatform = 'fantom'
        if chain == 'polygon': assetplatform = 'polygon-pos'

        price = cg.get_coin_market_chart_from_contract_address_by_id(id=assetplatform,
                                                                     contract_address=insider_contractAddress,
                                                                     vs_currency='usd',
                                                                     days='max')['prices']

        df = pd.DataFrame(price, columns=['timestamp', 'usd'])
        df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms', origin='unix')
        df['insider_contractAddress'] = insider_contractAddress
        df['insider_tokenSymbol'] = insider_tokenSymbol
        df['insider_tokenName'] = insider_tokenName
        df.to_csv(f'{WD}/price/csv/{insider_contractAddress}.csv', index=False)
        token.loc[token['insider_tokenSymbol'] == insider_tokenSymbol, 'status'] = 0
        token.to_csv(f'{WD}/price/list/token_list_complete.csv', index=False)
        
        dfs.append(df)
        print(f'{insider_tokenSymbol} done.')
        
    except Exception as e:
        time.sleep(2)
        print(f'{insider_tokenSymbol} contract address not found: {e}')
        dft = coin.loc[coin['Symbol'] == insider_tokenSymbol.lower()]
        dft = dft.loc[[True if jellyfish.jaro_winkler_similarity(i.lower(), insider_tokenName.lower()) >= 0.9 else False for i in dft['Id']]]
        if dft.shape[0] != 1: 
            length = dft.shape[0]
            print(f'  {insider_tokenSymbol} no outcome or several outcomes: {length}')
            continue
        try:
            Id = dft['Id'].values[0]
            price = cg.get_coin_market_chart_by_id(id=Id,
                                                   vs_currency='usd',
                                                   days='max')['prices']
            df = pd.DataFrame(price, columns=['timestamp', 'usd'])
            df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms', origin='unix')
            df['insider_contractAddress'] = insider_contractAddress
            df['insider_tokenSymbol'] = insider_tokenSymbol
            df['insider_tokenName'] = insider_tokenName
            df.to_csv(f'{WD}/price/csv/{insider_contractAddress}.csv', index=False)
            token.loc[token['insider_tokenSymbol'] == insider_tokenSymbol, 'status'] = 1
            token.to_csv(f'{WD}/price/list/token_list_complete.csv', index=False)
            dfs.append(df)
            print(f'  {insider_tokenSymbol} done.')
            
        except Exception as e:
            time.sleep(5)
            print(f'  {insider_tokenSymbol} coingecko symbol not found: {e}')
            
        
dfs = pd.concat(dfs)
dfs

TKING done.
TRIBE contract address not found: {'error': 'Could not find coin with the given id'}
  TRIBE done.
INCH contract address not found: {'error': 'Could not find coin with the given id'}
  INCH no outcome or several outcomes: 0
BITA done.
TSGO contract address not found: {'error': 'Could not find coin with the given id'}
  TSGO no outcome or several outcomes: 0
WOO done.
CR7 contract address not found: {'error': 'Could not find coin with the given id'}
  CR7 no outcome or several outcomes: 0
RAINI done.
XFT done.
ENS done.
BYTE contract address not found: {'error': 'Could not find coin with the given id'}
  BYTE no outcome or several outcomes: 0
SQUAWK done.
QOM done.
Squelch contract address not found: {'error': 'Could not find coin with the given id'}
  Squelch no outcome or several outcomes: 0
WILD done.
PLE done.
GRT done.
FABRIC done.
RND done.
nomad contract address not found: {'error': 'Could not find coin with the given id'}
  nomad no outcome or several outcomes: 0
SHP

,timestamp,usd,insider_contractAddress,insider_tokenSymbol,insider_tokenName
0,2021-05-18 00:00:00,0.00000558,0x24e89bdf2f65326b94e36978a7edeac63623dafa,TKING,Tiger King
1,2021-05-19 00:00:00,0.00000558,0x24e89bdf2f65326b94e36978a7edeac63623dafa,TKING,Tiger King
2,2021-05-20 00:00:00,0.00000296,0x24e89bdf2f65326b94e36978a7edeac63623dafa,TKING,Tiger King
3,2021-05-21 00:00:00,0.00000261,0x24e89bdf2f65326b94e36978a7edeac63623dafa,TKING,Tiger King
4,2021-05-22 00:00:00,0.00000172,0x24e89bdf2f65326b94e36978a7edeac63623dafa,TKING,Tiger King
...,...,...,...,...,...
201,2022-08-17 00:00:00,0.00317713,0x0d0B8488222F7f83B23E365320a4021b12eAD608,NXTT,NextEarthToken
202,2022-08-18 00:00:00,0.00282683,0x0d0B8488222F7f83B23E365320a4021b12eAD608,NXTT,NextEarthToken
203,2022-08-19 00:00:00,0.00296107,0x0d0B8488222F7f83B23E365320a4021b12eAD608,NXTT,NextEarthToken
204,2022-08-20 00:00:00,0.00279536,0x0d0B8488222F7f83B23E365320a4021b12eAD608,NXTT,NextEarthToken
